### CH16. RNN과 Attention을 이용한 자연어 처리
1. 문장에서 다음 글자를 예측하도록 훈련하는 문자 단위 RNN에 대해서 알아본다.
2. 새로운 텍스트를 생성하고 그 과정에서 매우 긴 sequence를 가진 tensorflow dataset을 만든다.
3. 상태가 없는 RNN을 사용하고 그것을 이용해서 상태가 있는 RNN을 구축한다.
    - 이때 상태가 없는 RNN은 각 반복에서 무작위하게 택한 텍스트의 일부분으로 학습하고 나머지 텍스트는 어떤 정보도 사용하지 않는다.
    - 상태가 있는 RNN은 훈련 반복 사이에 은닉 상태를 유지하고 중지된 곳에서 이어서 상태를 반영한다.
4. 감정 분석을 수행하는 RNN을 구축하고 단어 단위의 시퀀스로 문장을 다운다.
5. 신경망 기계번역은 encoder-decoder구조를 만든다.
6. Attention Mechanism을 이용해서 각 timestep에서 모델이집중해야 할 입력 부분을 선택하도록 학습한다.
7. RNN을 모두 제거하고 attention만을 이용해서 성능을 내는 transformer이라는 구조를 만든다.

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random

### 1. Char-RNN을 이용하여 셰익스피어 문체 생성하기
- Char-RNN을 이용하면 문장에서 한 번에 한 글자씩 새로운 텍스트를 생성할 수 있다.
- 즉, 문장에서 다음 단어를 예측하는 RNN을 설계하고 학습할 예정이다.

In [3]:
shakespeare_url = "https://homl.info/shakespeare"
filepath = tf.keras.utils.get_file("shakespeare.txt", shakespeare_url)

1122304/1115394 [==============================] - 0s 0us/step


In [4]:
with open(filepath) as f:
    shakespeare_text = f.read()

In [8]:
shakespeare_text[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

- 셰익스피어 dataset을 keras함수를 이용해서 불러왔으니 이제는 모든 글자를 정수로 인코딩하는 과정을 거쳐야 한다.
- 이를 위해서 keras의 Tokenizer class를 이용하고자 한다.
    - RNN모델에 텍스트 데이터를 학습 시키기 위해서는 텍스트에서 사용될 모든 단어를 각기 다른 ID로 mapping해 주어야 하기 때문이다.

In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(char_level = True)
tokenizer.fit_on_texts(shakespeare_text)

- ```char_level = True```로 설정하여 단어 수준 인코딩이 아닌 **글자 수준 인코딩**을 만든다.
- Tokenizer을 이용하면 기본적으로 text를 소문자로 바꾸어 주는데, 이게 필요 없다면 lower = False로 설정해 주면 된다.

In [12]:
tokenizer.texts_to_sequences(["First"])

[[20, 6, 9, 8, 3]]

In [13]:
max_id = len(tokenizer.word_index)
dataset_size = tokenizer.document_count

In [14]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text]))-1
#1을 빼준 이유는 id의 시작 숫자를 0으로 바꾸어 주기 위해서이다.

In [17]:
len(encoded)

1115394

- 이제 글자 단위로 나누어진 shakespeare_text dataset을 test, train, validation dataset으로 나누어 주어야 한다.
- 그러나 주의할 점은 이 데이터가 순차적인 데이터이기 때문에 순서가 바뀌어서는 안된다는 것이다.
- 시계열 데이터를 훈련, 검증, 테스트 세트로 나누는 것은 시간 순서로 나눈다 하더라도 변동이 생각보다 크기 때문에 쉬운 작업이 아니지만, ㅈ어진 문제에 따라 나누는 방법은 다양할 수 있다.

In [18]:
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

- 이 dataset을 이용해서 직접 신경망을 훈련시키면 결국에 RNN이 백만개의 층이 있는 심층 신경망과 비슷하고 매우 긴 sample로 훈련하는 셈이 된다.
- 그렇기 때문에 dataset의 window() 메서드를 이용해서 이 시퀀스를 작은 많은 텍스트 window로 바꾸어 주어야 한다. 
- 이렇게 하면 이 dataset의 각 sample은 전체 text data에서 매우 짧은 부분 문자열이 되고 RNN은 부분 문자열의 길이만큼 역전파를 위해 펼쳐진다.
    1. A "window" is a finite dataset of flat elements of size size (or possibly fewer if there are not enough input elements to fill the window and drop_remainder evaluates to False).

    2. The shift argument determines the number of input elements by which the window moves on each iteration. If windows and elements are both numbered starting at 0, the first element in window k will be element k * shift of the input dataset. In particular, the first element of the first window will always be the first element of the input dataset.

    3. The stride argument determines the stride of the input elements, and the shift argument determines the shift of the window.

In [20]:
n_steps = 100
window_length = n_steps + 1 #target = 1글자 앞의 input
dataset = dataset.window(window_length, shift = 1, drop_remainder = True)

- ```shift = 1```로 설정하면 0-100, 1-101이런 식으로 데이터를 window로 바꾸기 떄문에 가장 큰 훈련 세트를 만들 수 있다.
- padding을 하지 않고도 batch data를 만들도록 하기 위해서 모든 window가 동일하게 101개, 즉 window_length만큼의 글자를 포함하도록 drop_remainder = True로 설정한다.
- window() method를 이용하면 각각 하나의 dataset으로 표현이 되는 window를 포함하는 dataset을 만든다.
    - 이는 리스트의 리스트와 비슷한 nested dataset(중첩 데이터셋)이다. 이런 구조는 데이터셋을 섞거나 batch를 만드는데에 유용하지만 RNN모델을 tensor을 원하기 때문에 훈련에 이를 바로 이용할 수 없다.
    - 따라서 이 중첩 데이터셋을 flat dataset(데이터셋이 들어 있지 않은 데이터셋)으로 바꾸어 주어야 한다. 
        - 이를 위해 ```flat_map()```메서드를 이용한다.
        - 이 메서드는 중첩 데이터셋을 평평하게 만들기 전에 각 데이터셋에 적용할 변환함수를 매개변수로 받는다.

In [21]:
dataset = dataset.flat_map(lambda window:window.batch(window_length))

In [23]:
BATCH_SIZE = 32
BUFFER_SIZE = 1000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
dataset = dataset.map(lambda x:(x[:, :-1], x[:, 1:]))

**위의 방법으로 데이터셋을 섞고 batch를 분리해 준 뒤에 입력 데이터와 target을 구분해 주었는데, 입력데이터는 마지막 한 글자를 제외하고 입력하고 target은 마지막 한 글자를 포함한다.**

- 모델에 입력하기 위해서는 범주형 입력 특성은 무조건 원-핫 벡터나 Embedding layer을 이용해서 encoding을 해야 한다.
- 여기서는 고유한 글자수가 적기 때문에 Embedding Layer 대신 one-hot을 이용할 예정이다.
```
tf.one_hot(
    indices, depth, on_value=None, off_value=None, axis=None, dtype=None, name=None
)```  

indices:	A Tensor of indices.  (반드시 tensor의 형태를 입력해야 한다. 즉, tf.data.Dataset 처리가 된 데이터를 이용)  
depth:	A scalar defining the depth of the one hot dimension.  
on_value:	A scalar defining the value to fill in output when indices[j] = i. (default: 1)  
off_value:	A scalar defining the value to fill in output when indices[j] != i. (default: 0)  
axis:	The axis to fill (default: -1, a new inner-most axis).  
dtype:	The data type of the output tensor.  

In [26]:
dataset = dataset.map(lambda x_batch, y_batch: (tf.one_hot(x_batch, depth = max_id), y_batch)) 

In [27]:
dataset = dataset.prefetch(1)

- 이제 모델을 만들어 주면 된다.
- 텍스트에 있는 고유한 글자의 개수는 39개이기 떄문에 Time Distributed 클래스를 적용한 Dense layer의 경우에는 unit의 개수로 39개를 가짐으로서 각 timestep마다 각 글자에 대한 확률을 예측 할 수 있다.
- timestep마다 출력하는 확률의 합은 1이기 때문에 sparse_categorical_crossentropy를 손실함수로 사용하고 기울기와 편차같은 variable을 갱신하는 optimizer함수로는 Adam을 이용할 것이다.

In [28]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.GRU(128, return_sequences = True, input_shape = [None,max_id], dropout = 0.2, recurrent_dropout = 0.2))
model.add(tf.keras.layers.GRU(128, dropout = 0.2, recurrent_dropout = 0.2, return_sequences = True))
model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units = max_id, activation = 'softmax')))

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, None, 128)         64896     
_________________________________________________________________
gru_1 (GRU)                  (None, None, 128)         99072     
_________________________________________________________________
time_distributed (TimeDistri (None, None, 39)          5031      
Total params: 168,999
Trainable params: 168,999
Non-trainable params: 0
_________________________________________________________________


In [30]:
model.compile(loss= 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [31]:
model.fit(dataset, epochs = 5)

Epoch 1/5
31368/31368 [==============================] - 16989s 542ms/step - loss: 1.4767 - accuracy: 0.5465
Epoch 2/5
 7324/31368 [======>.......................] - ETA: 3:37:48 - loss: 1.3872 - accuracy: 0.5697

KeyboardInterrupt: 

In [32]:
def preprocess(texts):
    x = np.array(tokenizer.texts_to_sequences(texts))-1
    return tf.one_hot(x, depth = max_id)

In [35]:
x_new = preprocess(['How are yo'])
y_pred = model.predict_classes(x_new)
tokenizer.sequences_to_texts(y_pred+1)[0][-1]
#위와 같이 [0][-1]으로 slicing 하는 이유는 target으로 설정해 준것이 input_sequence보다 앞글자가 없고 마지막 글자만 추가된 sequence이기 떄문이다.
#따라서 예측 데이터또한 그렇게 100글자 짜리 일 것이고 우리가 알고 싶은 것은 마지막 글자이기 떄문이다.

'u'

- 이제 가짜 shakespeare text를 생성하도록 해보자.
    - 이를 하기 위해서는 먼저 초기 text를 주입하고 모델이 가장 가능성 있는 다음 단어를 선택하면 이를 텍스트에 이어 붙여서 또 모델에 주입하는 방법을 반복하면 된다.
    - 그러나 실제로는 이렇게 하면 같은 단어가 계속 반복되는 경우가 많다. 대신 ```tf.random.categorical()```함수를 사용해서 모델이 추정한 확률을 기반으로 다음 글자를 무작위로 선택하는 것이 가능하다.
        - 이를 할 때에 생성되는 글자의 다양성을 제어, 즉 더 높은 확률을 가지는 글자를 선택해 주기 위해서는 temperature(온도)라고 불리는 값을 지정하여서 이 값이 0에 가까울 수록 더 높은 확률의 글자를 선택하게 된다.
       

In [40]:
def next_char(text, temperature = 1):
    x_new = preprocess([text])
    y_pred = model.predict(x_new)[0,-1:,:]
    rescaled_logits = tf.math.log(y_pred) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples = 1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

def complete_text(text, n_chars = 50, temperature = 1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text
    

In [43]:
print(complete_text("a", temperature = 1))

are what not for o'er- make upon your lord!

hastin


#### 상태가 있는 RNN으로 분석하기
- 지금까지 사용한 RNN은 상태가 없는 RNN이었다. 그렇기 때문에 훈련 반복마다 모델의 은닉 상태를 0으로 초기화하고 time step마다 이 상태를 update한 뒤에 마지막 time step이후에는 버렸던 것이다.
- 그러나 RNN이 한 훈련 batch를 처리한 뒤에 마지만 상태를 다음 훈련 step의 초기 상태로 설정하게 된다면 역전파는 짧은 sequence에서 일어나지만 장기간 pattern을 학습할 수 있게 된다. (어떻게 보면 LSTM layer의 원리와도 비슷하다.)
    - 상태가 있는 RNN을 만들기 위해서는 데이터가 이전에 shift = 1로 설정한 것 처럼 겹치면 안된다.
    - 또한 겹치지 않기 위해서 하나의 window를 갖는 batch를 만들어야 한다.

In [45]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift = n_steps, drop_remainder = True)
dataset = dataset.flat_map(lambda window : window.batch(window_length))
dataset = dataset.batch(1)
dataset = dataset.map(lambda window : (window[:, :-1], window[:, 1:]))
dataset = dataset.map(lambda x,y: (tf.one_hot(x, depth = max_id), y))
dataset = dataset.prefetch(1)

**아래의 코드와 같은 방법으로 batch_size를 설정해서 dataset을 batch를 포함하도록 하면 한 batch에서 n번째 입력 sequence의 시작은 이전 batch의 n번째 입력 sequence의 마지막 부분이 된다.**

In [47]:
batch_size = 32
#겹치는 텍스트가 없는 batch를 만들기 위해서 batch_size개, 즉 32개로 train에 사용될 데이터를 분리해 준다.
encoded_parts = np.array_split(encoded[:train_size], batch_size)
#이후에 각각의 batch에 대해서 window_length로 배치를 위에서 긴시퀀스 전체에 한 것 처럼 진행을 한다.
datasets = []
for part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(part)
    dataset = dataset.window(window_length, shift = n_steps, drop_remainder = True)
    dataset = dataset.flat_map(lambda window : window.batch(window_length))
    datasets.append(dataset)
#datsets라는 리스트에 모든 batch dataset을 append한 뒤에 아래 코드를 이용해서 이를 연속적인 batch data로 만들어 준다.
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows:tf.stack(windows))

#그리고 다시 위에서 한 것 처럼 x_batch, y_batch를 나누어 주고 one_hot encoding을 진행한다.
dataset = dataset.map(lambda window : (window[:,:-1], window[:, 1:]))
dataset = dataset.map(lambda x,y : (tf.one_hot(x, max_id), y))
#마지막으로 prefetch까지 수행해 주면 dataset이 완성이 된다.
dataset = dataset.prefetch(1)

In [52]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.GRU(128, return_sequences = True, dropout = 0.2, recurrent_dropout = 0.2, stateful = True, batch_input_shape = [batch_size, None, max_id]))
model.add(tf.keras.layers.GRU(128, return_sequences = True, dropout = 0.2, recurrent_dropout = 0.2, stateful = True))
model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units = max_id, activation = 'softmax')))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_6 (GRU)                  (32, None, 128)           64896     
_________________________________________________________________
gru_7 (GRU)                  (32, None, 128)           99072     
_________________________________________________________________
time_distributed_2 (TimeDist (None, None, 39)          5031      
Total params: 168,999
Trainable params: 168,999
Non-trainable params: 0
_________________________________________________________________


In [53]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(dataset, epochs = 5)

Epoch 1/5
313/313 [==============================] - 140s 449ms/step - loss: 2.6244 - accuracy: 0.2588
Epoch 2/5
313/313 [==============================] - 155s 495ms/step - loss: 2.1955 - accuracy: 0.3548
Epoch 3/5
313/313 [==============================] - 159s 508ms/step - loss: 2.0621 - accuracy: 0.3885
Epoch 4/5
313/313 [==============================] - 159s 507ms/step - loss: 2.0329 - accuracy: 0.3981
Epoch 5/5
313/313 [==============================] - 159s 508ms/step - loss: 2.0081 - accuracy: 0.4060


#### 지금까지는 글자 수준의 모델을 만들었고, 이제는 단어 수준 모델을 살펴 보면서 자주 등장하는 자연어 처리 작업인 sentiment analysis를 다루어 볼 차례이다.   
#### 뿐만 아니라 이 과정에서 masking을 사용해서 길이가 다른 sequence를 다루는 방법을 알아볼 예정이다.